# Real-time Object Detection with TensorFlow and OpenCV for Dragline Safety

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
# Load pre-trained MobileNetV2 model for object detection
model = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=True)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
# Load pre-trained object detection model from TensorFlow Hub
detector = cv2.dnn.readNetFromTensorflow(
    'ssd_mobilenet_v2_coco/frozen_inference_graph.pb',
    'ssd_mobilenet_v2_coco/coco.names'
)

In [ ]:
def detect_objects(frame):
    # Resize and preprocess the frame
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    detector.setInput(blob)
    detections = detector.forward()

    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:  # Filter out weak detections
            class_id = int(detections[0, 0, i, 1])
            label = f"Class: {class_id}"

            # Draw bounding box and label on the frame
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame
# Capture video from the webcam
cap = cv2.VideoCapture(0)


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects in the frame
    output_frame = detect_objects(frame)

    # Display the output frame
    cv2.imshow("Object Detection Demo", output_frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()